## Retrieve electric vehicle charging locations using the NREL API

---
**OBJECTIVES**
 * Understand what an API is and why it's useful
 * Hands on experience using Python to extract data via an API
 * Review of Pandas techniques for handling tabular data, including export
---

### ♦ What is an API?
Every time you browse a web site, you are sending a **request** from your browser (or **client**) to a remote **server** to send back the **content** in a format that your browser can interpret and display. Often, the actual request you send can be seen in the web site's URL, which includes the address of the server, the service name, and parameters for the service. 

> **Try this**:
> * Open Google's home page: https://google.com
> * Search for "Nicholas School of the Environment" 
> * When complete, look at the web address of the search results page. 
> * Simplified, the address is https://google.com/search?q=Nicholas+School+of+the+Environment. 
> * Can you identify the `service address`, `service name`, and `parameters` in the above URL? 

APIs, or Application Programming Interfaces, are the set of services and parameters that a particular client (e.g. Google) provides, and they are the basis for a lot of client/server or "cloud" computing. 


### ♦ How to use an API
Many data portals have APIs that allow us to request specific datasets and download them to our local machine. Here we examine some of NREL's API and how we can use these APIs to extract electric vehicle charging locations as a CSV file with coordinates. To use this API, you'll need to sign up for an API key - a unique identifier that allows NREL to track usage and ensure that no single user is overloading the service with requests.

> **Explore the NREL API**:
> * Navigate to the NREL API main site: https://developer.nrel.gov/
> * Examine the various themese in which APIs are provided. **Then sign up for a key.** <br>→*Create a new text document called `NRELkey.txt` and save your key in that file.* 
> * Navigate to the group of transportation related APIs: https://developer.nrel.gov/docs/transportation/
> * And from there, navigate to APIs on [alternative fuel stations](https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/)

You see they have various ways of extracting stations. You can just ask to get all stations, a specific station, or even do some rudimentary spatial analysis to filter stations geographically. So let's look at how to engage these APIs. 

### ♦ Python packages used to retrive data via APIs
We'll use the Python `requests` package to send our request and handle the response (i.e. the data returned). And we'll use `Pandas` to wrangle the data returned into a nicely formatted dataframe and export as a tidy CSV file. 

### Code to request, process, and save data using the NREL API
The steps invovled in invoking an API in Python include: 
* Importing the packages used to handle the API request ('requests') and to handle the data ('pandas').
* Constructing the request, adhering to the API documentation
* Sending the request and accepting the raw response 
* Wrangling the raw response into a useful data object
* Saving the data in a useful format (e.g. CSV) to a local file. 

In [ ]:
#Import requests and pandas packages to our Python environment
import requests
import pandas as pd

*You can either set the `key` variable equal to your API key. Or what I like to do is save it to a text file (mine is called `NRELkey.txt`) and then read the value from the text file into the script. That way the key does not appear in your code...*

In [ ]:
#Read key file in
key = open('NRELkey.txt','r').readline()

First, we'll use the `All Stations` api to extract a set of sites. 
* View the [documentation](https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/) for this API.
* Note the Request URL: `GET /api/alt-fuel-stations/v1.format?parameters`. This indicates how the API call is structured. 
* Also note what parameters we can use and how they are called. 

Below, we'll construct our request in two parts. The first part will be the `baseURL`; this includes the web address (`https://developer.nrel.gov/api/`) and service name (`alt-fuel-stations/v1.json`). The second part will be a Python *[dictionary object](https://www.w3schools.com/python/python_dictionaries.asp)* containing the set of parameter names and values we want to include in our request. 

In [ ]:
#Construct requests to fetch all stations in North Carolina
baseURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?'
params = {
    'api_key':key,                #Our key, to authenticate the service
    'status':'E',                 #Request only currently open sites 
    'fuel_type':'ELEC',           #Request only Electric sites
    'ev_charging_level':'dc_fast',#Request only DCFC sites
    'state':'NC' ,                #Request only NC sites
    'limit':'all'                 #Don't cap results to first 200 (default)
}

---
<div class="alert alert-info">
  ► TASK: Modify the above code cell so that only public sites are requested
</div> 

In [ ]:
#TASK: Modify the above code cells to return only public sites
baseURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?'
params = {
    'api_key':key,                #Our key, to authenticate the service
    'status':'E',                 #Request only currently open sites 
    'fuel_type':'ELEC',           #Request only Electric sites
    'ev_charging_level':'dc_fast',#Request only DCFC sites
    'state':'NC' ,                #Request only NC sites
    'limit':'all'                 #Don't cap results to first 200 (default)
}

---
Now, with our request objects created, we use the `requests.get` function to send that request off to the NREL server, saving the server's response as the variable `raw_result`

In [ ]:
#Execute request, storing result as the variable "raw_result"
result_raw = requests.get(baseURL,params)

In [ ]:
#What kind of Python object is the `raw_result`? Use the "type()" command to find out...
type(result_raw)

In [ ]:
#What can I do with this raw_result object? Use the "help()"" command to find out...
help(result_raw)

In [ ]:
#What does the "json" method do? Use the Juptyer help ("?") to find out...
?result_raw.json

We'll convert our "raw" response object to a **JSON** formatted object, which translates into a **Python dictionary** object. 

In [ ]:
#Convert the raw_result object to a JSON dictionary
result_json = result_raw.json()

In [ ]:
#What is this json object?
type(result_json)

The dictionary is hierarchical, with different response **values** stored with different **keys**, so let's see what those keys are named...

In [ ]:
#Examine the keys associated with the dictionary
result_json.keys()

The dictionary keys correspond with the ["response fields" listed in the API's documentation](https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/#response-fields).

The values associated with the `fuel_stations` key holds the listing all the results...

In [ ]:
#Examine the values associated with the "fuel_stations" key in the result_json dictionary
result_json['fuel_stations']

Next, we convert this dictionary of values to a **Pandas dataframe**. Pandas is a powerful package that allows a great deal of data exploration, visualization, analysis, and transformation. We'll just examine a few features.

In [ ]:
#Pull results in 'fuel_stations' item into a dataframe
df = pd.DataFrame.from_dict(result_json['fuel_stations'])

In [ ]:
#Reveal the size of this dataframe, in rows and columns
df.shape

In [ ]:
#Reveal the columns associated with the data
df.columns

We have a lot of extraneous columns in our table. Below is code to identify those matching specific criteria and then remove them.

In [ ]:
#Create a list of non EV columns
dropCols = [col for col in df.columns if col[:3] in ('bd_','cng','hy_','ng_','e85','lng','lpg')]
#Append a list French columns to the above list
dropCols += [col for col in df.columns if col[-2:] == 'fr']
#Print the list
print(dropCols)

In [ ]:
#Drop the columns identified above
df.drop(columns = dropCols,inplace=True)

In [ ]:
#Examine the column list again
df.columns

In [ ]:
#Reveal the first 5 rows of data
df.head()

In [ ]:
#Look at unique values in the "access_days_time" field
df['access_days_time'].unique()

In [ ]:
#Plot, using matplotlib
%matplotlib inline
df.plot(x='longitude',
        y='latitude',
        kind='scatter',
        grid=True);

And finally, we'll export our Pandas dataframe to a CSV format file...

In [ ]:
#Export data to csv file, omitting the index
df.to_csv("NREL_DFCFsites.csv",index=False)